<img src="./Logo_UNSAM.png" align="right" width="150" />

#### Procesamiento Digital de Señales
# TP final

## Detección de fallas mecánicas en maquinaria industrial mediante análisis de señales acústicas (MIMII Dataset)


#### Lucas Marú


# Introducción

En entornos industriales, el mantenimiento predictivo cumple un rol fundamental para evitar fallas inesperadas, reducir costos de reparación y optimizar la disponibilidad de los equipos. Entre las estrategias disponibles, el análisis de señales acústicas ha demostrado ser una herramienta efectiva para anticipar fallos a partir de cambios en el sonido emitido por las máquinas durante su funcionamiento.

El problema que se aborda en este trabajo es la detección de fallas en maquinaria a partir del sonido. Específicamente, se busca observar diferencias entre el comportamiento acústico de una máquina funcionando correctamente y la misma máquina operando en condiciones anómalas. Esta aproximación es especialmente relevante cuando no se dispone de sensores internos o cuando se desea realizar un monitoreo no invasivo.

Para llevar adelante este análisis se utilizará el MIMII Dataset (Malfunctioning Industrial Machine Investigation and Inspection), una base de datos pública que contiene grabaciones de audio de cuatro tipos de máquinas industriales: ventiladores, bombas, válvulas y rieles deslizantes. Las grabaciones incluyen condiciones normales y con fallas simuladas, y están disponibles en formato .wav, multicanal (8 micrófonos) y en distintos niveles de ruido ambiental para simular entornos realistas.

A pesar de que el MIMII Dataset fue diseñado principalmente para ser utilizado en el desarrollo y entrenamiento de modelos de detección de anomalías basados en aprendizaje profundo, en este trabajo se propone un enfoque distinto, más artesanal y centrado en la exploración. La idea es aplicar herramientas de procesamiento de señales estudiadas durante la cursada —como la transformada de Fourier, la estimación espectral con Welch y el análisis de energía— para identificar patrones o diferencias entre señales normales y anómalas. Esta decisión metodológica responde tanto al interés por comprender el comportamiento acústico de las máquinas como al alcance técnico de la materia.



# Objetivos

### Objetivo general  
Analizar señales acústicas de una máquina industrial (ventilador) mediante herramientas clásicas de procesamiento de señales, con el fin de detectar posibles diferencias entre condiciones normales y defectuosas, sin recurrir a modelos de aprendizaje automático.

### Objetivos específicos
- Visualizar y comparar señales individuales en el dominio temporal y frecuencial.
- Estimar el contenido espectral mediante FFT y Welch.
- Calcular métricas como energía por banda y detectar picos espectrales.
- Construir un perfil espectral típico de señales normales y compararlo contra señales anómalas.
- Explorar si es posible detectar señales atípicas a partir de su desviación respecto al comportamiento normal.


# Descripción del dataset

Se trabajará con un subconjunto del **MIMII Dataset**, una base de datos pública que contiene grabaciones de audio de cuatro tipos de máquinas industriales: ventiladores, bombas, válvulas y rieles deslizantes. Cada tipo incluye siete máquinas individuales (`id_00`, `id_02`, etc.), registradas tanto en condiciones normales como con fallas simuladas, y en diferentes niveles de ruido ambiental (6 dB, 0 dB, -6 dB).

Las señales están almacenadas en archivos `.wav` de 10 segundos, grabados en simultáneo con 8 micrófonos a una frecuencia de muestreo de 16 kHz. Para mantener el análisis acotado y manejable, en esta etapa se trabajará exclusivamente con uno de los modelos disponibles: `fan_id_00`. Se analizarán tanto señales mono canal (utilizando un solo micrófono) como señales promediadas entre canales, con el objetivo de evaluar si el comportamiento acústico característico se mantiene o se refuerza según la forma de representación.

### Observaciones técnicas

Las señales anómalas fueron registradas tras introducir fallas reales (como desbalanceo o contaminación) en condiciones controladas, aunque el tipo de falla no se encuentra especificado. Esto refuerza el enfoque general del dataset, orientado a la detección de desviaciones respecto al comportamiento normal.

Las versiones con diferentes SNR fueron generadas mezclando las señales originales con ruido ambiental real, escalado para alcanzar una relación señal/ruido predeterminada. El contenido de la señal de máquina se mantiene constante entre versiones, lo que permite estudiar la influencia del ruido de forma aislada.

A continuación se muestra un resumen del contenido total del MIMII Dataset por tipo de máquina y modelo (id), separado por condiciones normales y anómalas, lo cual permite visualizar de forma clara la composición completa del conjunto de datos:

<p align="center">
  <img src="Tabla_dataset.png" alt="Tabla MIMII" width="350">
</p>

*Figura – Cantidad de segmentos por condición y modelo, tomada de Purohit et al., 2019.*

